In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [2]:
#zero initialization
neuron = Neuron(2,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [3]:
#random initialization
neuron = Neuron(2,identity,method_ini = "RandomNor", random_mean = -1000.0, random_std = 10.0)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [4]:
#initialization from list
vector = [1.0,2.0,3.5]
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [6]:
#initialization from row vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_row = vector[np.newaxis,:]
neuron = Neuron(vector_row,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [7]:
#initialization from column vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_column = vector[:,np.newaxis]
neuron = Neuron(vector_column,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [8]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [9]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [10]:
#initialization by number of dim in list
dim_layer = [4,5]
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [11]:
#initialization by giving weight array
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [12]:
#initialization by giving weight array and function (single) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,[identity],method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [13]:
#initialization by giving weight array and identity function (mutliple same) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [identity,identity,identity]
testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [14]:
#initialization by giving weight array and identity function (mutliple different) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [sigmoid,relu,leaky_relu]
testLayer = Layer(weights,activation_functions)
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [15]:
#error messages testing
print("Test for: wrong var type (not number) given in list for weight initialization")
try:
    vector = ["as","2.0"]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in list for weight initialization")
try:
    vector = [1,2.3]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type (not number) given in 1D array for weight initialization")
try:
    vector = np.array(["as","2.0"])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in 1D array for weight initialization")
try:
    vector = np.array([1,2.3])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong dimensions of given 1D array for weight initialization")
try:
    vector = np.array([1,2,1,1,2])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not rational numbers given in 2D array for weight initialization")
try:
    weights = np.array([[1.0 + 4j,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough information given in 2D array for weight initialization")
try:
    weights = np.array([[]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: bigger than 2D was given for array for weight initialization")
try:
    weights = np.array([[[2]]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not compatible amount of activation functions given to the amount of neurons for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,relu,leaky_relu,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (inside list) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,1,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (standalone) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = 1
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, too much dimensions in input")
try:
    #Input data for input propagation (forward pass)
    inputTestArray = np.array([[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]],[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, complex input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = np.array([[1.0 + 3j],[1.0],[1.0],[1.0],[1.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = "sda"
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input in list")
try:
    #Input data for input propagation (forward pass)
    inputTestList =["a",1.0,1.0,1.0,1.0]
    inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, input does not match layer neurons")
try:
    #Input data for input propagation (forward pass)
    inputTestList = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: wrong var type (not number) given in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type (not number) given in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weight

FNN class implementation testing

In [16]:
#initialization by number of dim in vector(list)
dim_layer = [5,5,3,2,1]
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list_list[0])
print(testNet.activ_functions_list_list[1])
print(testNet.activ_functions_list_list[2])
print(testNet.activ_functions_list_list[3])
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[[-0.82547822 -0.89294912 -1.69019465  0.14385809 -1.18034835 -1.99055712]
 [-0.96640674  0.79462566 -0.46674804  1.54673005  1.6409968   0.35427461]
 [ 0.33623754  0.37133223  0.43228008 -0.41554514 -0.04729416 -0.4013322 ]
 [-1.34909812  1.92605387  0.21312322  0.49625332 -1.04868976  0.27296564]
 [ 1.23057299  0.55143136 -1.10964164 -0.34438262 -1.26544343 -0.50294381]]
[[ 0.18140716 -0.55256905 -0.8541168  -0.4089883  -0.7790216  -0.90819353]
 [-1.11999626 -0.60930385 -0.06146198 -1.80689258  0.94695047  0.13023029]
 [ 1.15936889  0.31834151 -1.64817853 -0.40436634 -0.23484601 -1.51522973]]
[[-0.08460807  2.17618878 -0.4317899  -0.32520531]
 [ 1.07102834  0.10662526  0.35303526  0.33654854]]
[[-0.21382914  0.02920939 -0.93861506]]
[<function relu at 0x00000268FFF33920>, <function relu at 0x00000268FFF33920>, <function relu at 0x00000268FFF33920>, <function relu at 0x00000268FFF33920>, <function relu at 0x00000268FFF33920>]
[<function relu at 0x00000268FFF33920>, <function relu at 0

True

In [17]:
#initialization by number of dim in vector(1D array)
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [18]:
#dummy network ini
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#initialization by ready weight array
dim_layer = testNet.weights_list
print(dim_layer)
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[array([[ 0.69931229, -0.6348745 ,  0.72328942,  0.02097643, -0.10902594,
         0.02227184],
       [ 1.00832698,  0.78000946,  0.88849933, -1.75664204, -1.02947266,
        -0.39329653],
       [ 1.51385128,  0.47101717,  0.86374337,  0.9537957 , -0.58084912,
         0.79671417],
       [-0.10256125, -0.61249739,  0.71997166, -0.6811799 ,  0.76801229,
        -1.1973228 ],
       [-0.41738736,  0.74528934, -1.66500714,  1.27846166, -0.24140834,
         0.46710437]]), array([[-0.02734724,  1.56075694,  0.55677231,  1.02152029, -1.76070724,
        -0.25533182],
       [ 0.78645498,  1.30364105, -0.66873205,  1.36762582,  1.12260879,
        -0.72913749],
       [ 0.61933348,  1.89159381, -0.10068615,  1.41160327,  0.76046919,
        -0.0146493 ]]), array([[-1.65589093, -1.23940333,  1.17227549, -1.58772988],
       [ 0.57351154, -1.1565181 , -0.74323125, -0.6580493 ]]), array([[-0.2086174 , -1.47148676, -1.04636919]])]


True

In [19]:
#initialization by number of dim in vector(1D array) and 1 activation function(as value)
dim_layer = np.array([5,5,3,2,1])
activ_func = identity
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [20]:
#initialization by number of dim in vector(1D array) and 1 activation function(as list)
dim_layer = np.array([5,5,3,2,1])
activ_func = [identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [21]:
#initialization by number of dim in vector(1D array) and 2 activation functions
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [22]:
#initialization by number of dim in vector(1D array) and activation function for each layer
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,sigmoid,leaky_relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [23]:
#initialization by number of dim in vector(1D array) and activation function for each neuron
dim_layer = np.array([5,5,3,2,1])
l1 = [relu,sigmoid,sigmoid,leaky_relu,identity]
l2 = [relu,sigmoid,sigmoid]
l3 = [relu,sigmoid]
l4 = [identity]
activ_func =  [l1,l2,l3,l4]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [24]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Zero weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="Zero")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [25]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [26]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [27]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [28]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [29]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [30]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [31]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#decomposing network
layers_list = testNet.decomposeIntoLayers()
neurons_list_list = testNet.decomposeIntoNeurons()
#checking the results
print(layers_list)
print(neurons_list_list)

[<Layer.Layer object at 0x0000026898040150>, <Layer.Layer object at 0x0000026898041090>, <Layer.Layer object at 0x0000026898043F50>, <Layer.Layer object at 0x0000026898041890>]
[[<Neuron.Neuron object at 0x00000268980404D0>, <Neuron.Neuron object at 0x0000026898040690>, <Neuron.Neuron object at 0x0000026898040550>, <Neuron.Neuron object at 0x0000026898040590>, <Neuron.Neuron object at 0x0000026898040610>], [<Neuron.Neuron object at 0x0000026898040410>, <Neuron.Neuron object at 0x0000026898040450>, <Neuron.Neuron object at 0x0000026898040850>], [<Neuron.Neuron object at 0x0000026898040910>, <Neuron.Neuron object at 0x0000026898040950>], [<Neuron.Neuron object at 0x00000268980409D0>]]


In [32]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: not supported input for weights
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Not supported data type given.
Test for: array of not supported dimension
Correct errors was caught. Error:  Given array is of size not supported by implementation. Supported dimensions: 1.
Test for: given numbers are not integers
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Given values in vector must be integers to properly represent number of neurons in layers
Test for: not enough numbers given. Minimally required input and output layers cannot be created
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. At least two numbers are needed for FNN creation: FNN needs at least input and output layer (single layer network).


Reload Modules


In [33]:
import importlib
import sys
import numpy as np

# Reload order:
modules_in_order = [
    "ActivFunctions",
    "SuppFunctions",
    "InitFunctions",
    "Layer",
    "FNN",
    "TrainingFunctions"
]

for m in modules_in_order:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        globals()[m] = importlib.import_module(m)

print("All modules reloaded successfully.")


All modules reloaded successfully.


Imports

In [34]:
from FNN import FNN
from LossFunctions import *

# Activations
from ActivFunctions import *
import numpy as np


In [35]:
print("=== Test 1: Single-Layer Network Gradient ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVect = np.ones((4, 1))
targetVect = 0.5 * np.ones((5, 1))

out = testNet.forward(inputTestVect)
grad = testNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)


=== Test 1: Single-Layer Network Gradient ===

Gradient matrix for single layer:
[[ 0.87721862  0.87721862  0.87721862  0.87721862  0.87721862]
 [-2.45694047 -2.45694047 -2.45694047 -2.45694047 -2.45694047]
 [-0.15863644 -0.15863644 -0.15863644 -0.15863644 -0.15863644]
 [-0.72388618 -0.72388618 -0.72388618 -0.72388618 -0.72388618]
 [-5.15120186 -5.15120186 -5.15120186 -5.15120186 -5.15120186]]
[array([[ 0.87721862,  0.87721862,  0.87721862,  0.87721862,  0.87721862],
       [-2.45694047, -2.45694047, -2.45694047, -2.45694047, -2.45694047],
       [-0.15863644, -0.15863644, -0.15863644, -0.15863644, -0.15863644],
       [-0.72388618, -0.72388618, -0.72388618, -0.72388618, -0.72388618],
       [-5.15120186, -5.15120186, -5.15120186, -5.15120186, -5.15120186]])]


In [36]:
print("=== Test 1: Single-Layer Network Gradient with multi column input ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, softmax, method_ini="RandomNor")

inputTestVectt = np.ones((4, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = 0.5 * np.ones((5, 1))
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 
out = testNet.forward(inputTestArray)
grad = testNet.backward(out[0],out[1],targetTestArray,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)

=== Test 1: Single-Layer Network Gradient with multi column input ===

Gradient matrix for single layer:
[[-0.04088596 -0.04088596 -0.04088596 -0.04088596 -0.04088596]
 [-0.09561931 -0.09561931 -0.09561931 -0.09561931 -0.09561931]
 [ 0.06479182  0.06479182  0.06479182  0.06479182  0.06479182]
 [-0.01927879 -0.01927879 -0.01927879 -0.01927879 -0.01927879]
 [-0.07130593 -0.07130593 -0.07130593 -0.07130593 -0.07130593]]
[array([[-0.04088596, -0.04088596, -0.04088596, -0.04088596, -0.04088596],
       [-0.09561931, -0.09561931, -0.09561931, -0.09561931, -0.09561931],
       [ 0.06479182,  0.06479182,  0.06479182,  0.06479182,  0.06479182],
       [-0.01927879, -0.01927879, -0.01927879, -0.01927879, -0.01927879],
       [-0.07130593, -0.07130593, -0.07130593, -0.07130593, -0.07130593]])]


In [37]:
print("=== Test 2: Deep 10-Layer Network ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, [relu,softmax], method_ini="RandomNor")

inputVect = np.random.normal(1,10,(5, 1))
targetVect = np.array([[0.7]])

out = deepNet.forward(inputVect)
grad = deepNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)


=== Test 2: Deep 10-Layer Network ===

Gradient matrix for layer 0:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 1:
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 2:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 3:
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 4:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [38]:
print("=== Test 2: Deep 10-Layer Network with multi column input ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 5
])

deepNet = FNN(dim_layer, [relu,softmax], method_ini="RandomNor")

inputTestVectt =  np.ones((5, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = np.array([[1],[0],[1],[0],[1]])
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 

#inputVect = np.ones((5, 1))
#targetVect = np.array([[0.7]])

out = deepNet.forward(inputTestArray)
grad = deepNet.backward(out[0],out[1],targetTestArray,SoftmaxCrossEntropyDerivative)



for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)

=== Test 2: Deep 10-Layer Network with multi column input ===

Gradient matrix for layer 0:
[[ 1.21847914e-05  1.21847914e-05  1.21847914e-05  1.21847914e-05
   1.21847914e-05  1.21847914e-05]
 [ 1.82816940e-06  1.82816940e-06  1.82816940e-06  1.82816940e-06
   1.82816940e-06  1.82816940e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.16666027e-07  7.16666027e-07  7.16666027e-07  7.16666027e-07
   7.16666027e-07  7.16666027e-07]
 [-6.87315351e-06 -6.87315351e-06 -6.87315351e-06 -6.87315351e-06
  -6.87315351e-06 -6.87315351e-06]
 [-1.19840014e-05 -1.19840014e-05 -1.19840014e-05 -1.19840014e-05
  -1.19840014e-05 -1.19840014e-05]
 [ 3.12023390e-06  3.12023390e-06  3.12023390e-06  3.12023390e-06
   3.12023390e-06  3.12023390e-06]
 [-3.63237590e-06 -3.63237590e-06 -3.63237590e-06 -3.63237590e-06
  -3.63237590e-06 -3.63237590e-06]]

Gradient matrix for layer 1:
[[-1.80066098e-06 -2.32051242e-06 -2.66982216e-06  0.00000000e+00
  -